In [ ]:
!cp '/content/drive/MyDrive/HACKER EARTH/7342422066d511eb.zip'  .

In [ ]:
!unzip -q 7342422066d511eb.zip

replace dataset/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

path = os.getcwd()
data_path = '//content//dataset//'

In [ ]:
train = pd.read_csv(data_path + 'train.csv')
test  = pd.read_csv(data_path + 'test.csv')
sample_sub = pd.read_csv(data_path + 'sample_submission.csv')

In [ ]:
df = pd.concat([train, test], axis = 0).reset_index(drop=True)

In [ ]:
df.isnull().sum()

Customer Id                 0
Artist Name                 0
Artist Reputation         972
Height                    494
Width                     725
Weight                    736
Material                  764
Price Of Sculpture          0
Base Shipping Price         0
International               0
Express Shipment            0
Installation Included       0
Transport                1624
Fragile                     0
Customer Information        0
Remote Location           771
Scheduled Date              0
Delivery Date               0
Customer Location           0
Cost                     3500
dtype: int64

In [ ]:
df['Artist Reputation'].fillna(df['Artist Reputation'].median(), inplace = True)

In [ ]:
def height_imputer_by_material_weight(x):
    if x['Weight'] <= 1000:
        lower = 0
        upper = 999
    elif x['Weight']>1000 and x['Weight']<10000:
        lower = 1000
        upper = 10000
    elif x['Weight']>10000 and x['Weight']<100000:
        lower = 10000
        upper = 100000
    else:
        lower = 100000
        upper = 100000000000000000
    impute = df[ (df['Material'] == x['Material']) & (df['Weight']<=upper) & (df['Weight']>=lower)]['Height'].median()
    return impute

In [ ]:
df.loc[ (df['Height'].isna()) & (~df['Weight'].isna()) & (~df['Material'].isna()), 'Height'] = df[ (df['Height'].isna()) & (~df['Weight'].isna()) & (~df['Material'].isna())].apply(lambda x: height_imputer_by_material_weight(x), axis = 1)

In [ ]:
df['Height'].fillna(df['Height'].median(), inplace = True)

In [ ]:
def width_imputer_by_material_weight(x):
    if x['Weight'] <= 1000:
        lower = 0
        upper = 999
    elif x['Weight']>1000 and x['Weight']<10000:
        lower = 1000
        upper = 10000
    elif x['Weight']>10000 and x['Weight']<100000:
        lower = 10000
        upper = 100000
    else:
        lower = 100000
        upper = 100000000000000000
    impute = df[ (df['Material'] == x['Material']) & (df['Weight']<=upper) & (df['Weight']>=lower)]['Width'].median()
    return impute

In [ ]:
df.loc[ (df['Width'].isna()) & (~df['Weight'].isna()) & (~df['Material'].isna()), 'Width'] = df[ (df['Width'].isna()) & (~df['Weight'].isna()) & (~df['Material'].isna())].apply(lambda x: height_imputer_by_material_weight(x), axis = 1)

In [ ]:
df['Width'].fillna(df['Width'].median(), inplace = True)

In [ ]:
def weight_imputer_by_material_weight(x):
    if x['Height'] <= 10:
        lower = 0
        upper = 10
    elif x['Height']>10 and x['Height']<25:
        lower = 10
        upper = 25
    elif x['Height']>25 and x['Height']<50:
        lower = 25
        upper = 50
    else:
        lower = 50
        upper = 100
    if x['Width'] <= 10:
        lower_1 = 0
        upper_1 = 10
    elif x['Width']>10 and x['Width']<20:
        lower_1 = 10
        upper_1 = 20
    elif x['Width']>20 and x['Width']<35:
        lower_1 = 20
        upper_1 = 35
    else:
        lower_1 = 35
        upper_1 = 100
    impute = df[ (df['Width']<=upper_1) & (df['Width']>=lower_1) & (df['Height']<=upper) & (df['Height']>=lower)]['Weight'].median()
    return impute

In [ ]:
df.loc[ (df['Weight'].isna()), 'Weight'] = df[ (df['Weight'].isna())].apply(lambda x: weight_imputer_by_material_weight(x), axis = 1)

In [ ]:
df.loc[982, 'Weight'] = 160322.500
df.loc[1678, 'Weight'] = 160322.500

In [ ]:
def material_imputer_by_material_weight(x):
    if x['Height'] <= 10:
        lower = 0
        upper = 10
    elif x['Height']>10 and x['Height']<25:
        lower = 10
        upper = 25
    elif x['Height']>25 and x['Height']<50:
        lower = 25
        upper = 50
    else:
        lower = 50
        upper = 100
    if x['Width'] <= 10:
        lower_1 = 0
        upper_1 = 10
    elif x['Width']>10 and x['Width']<20:
        lower_1 = 10
        upper_1 = 20
    elif x['Width']>20 and x['Width']<35:
        lower_1 = 20
        upper_1 = 35
    else:
        lower_1 = 35
        upper_1 = 100
    if x['Weight'] <= 1000:
        lower_2 = 0
        upper_2 = 999
    elif x['Weight']>1000 and x['Weight']<10000:
        lower_2 = 1000
        upper_2 = 10000
    elif x['Weight']>10000 and x['Weight']<100000:
        lower_2 = 10000
        upper_2 = 100000
    else:
        lower_2 = 100000
        upper_2 = 100000000000000000
    dic = df[ (df['Width']<=upper_1) & (df['Width']>=lower_1) & (df['Height']<=upper) & (df['Height']>=lower) & (df['Weight']>=lower_2) & (df['Weight']>=upper_2)]['Material'].value_counts().to_dict()
    if len(dic) == 0:
        return None
    impute = list(dic.keys())[0]
    return impute

In [ ]:
df.loc[ (df['Material'].isna()), 'Material'] = df[ (df['Material'].isna())].apply(lambda x: material_imputer_by_material_weight(x), axis = 1)

In [ ]:
df['Material'].fillna('Stone', inplace = True)

In [ ]:
df.loc[ (df['Transport'].isna()) & (df['Express Shipment'] == 'Yes'), 'Transport'] = 'Waterways'

In [ ]:
df.loc[ (df['Transport'].isna()) & (df['Customer Information'] == 'Wealthy'), 'Transport'] = 'Airways'

In [ ]:
df['Transport'].fillna('Roadways', inplace = True)

In [ ]:
df.groupby('Customer Information')['Remote Location'].value_counts()

Customer Information  Remote Location
Wealthy               No                 1959
                      Yes                 477
Working Class         No                 5424
                      Yes                1369
Name: Remote Location, dtype: int64

In [ ]:
df.loc[ (df['Remote Location'].isna()) & (df['Customer Information'] == 'Wealthy'), 'Remote Location'] = 'No'
df['Remote Location'].fillna('Yes', inplace = True)


In [ ]:
train, test = df[: train.shape[0]], df[train.shape[0]:].reset_index(drop = True)

In [ ]:
data_path = '//content//drive//MyDrive//HACKER EARTH//'

In [ ]:
train.to_csv(data_path + 'cleaned_train.csv', index = False)
test.to_csv(data_path + 'cleaned_test.csv', index = False)